# TensorBoard Basics

In [1]:
#More stuff here: https://blog.jakuba.net/2017/05/30/tensorflow-visualization.html

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

import tensorflow as tf

# Set parameters

learning_rate = 0.01
training_iteration = 30
batch_size = 100
display_step = 2


ModuleNotFoundError: No module named 'tensorflow.contrib'

In [2]:
# TF graph input
x = tf.placeholder("float", [None, 784]) # mnist data image of shape 28*28=784
y = tf.placeholder("float", [None, 10]) # 0-9 digits recognition => 10 classes


In [3]:
# Create a model

# Set model weights
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

with tf.name_scope("Wx_b") as scope:
    # Construct a linear model
    model = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
    
# Add summary ops to collect data
w_h = tf.summary.histogram("weights", W)
b_h = tf.summary.histogram("biases", b)

In [4]:
# More name scopes will clean up graph representation
with tf.name_scope("cost_function") as scope:
    # Minimize error using cross entropy
    # Cross entropy
    cost_function = -tf.reduce_sum(y*tf.log(model))
    # Create a summary to monitor the cost function
    tf.summary.scalar("cost_function", cost_function)

In [5]:
with tf.name_scope("train") as scope:
    # Gradient descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

In [7]:
# Initializing the variables
init = tf.global_variables_initializer()

# Merge all summaries into a single operator
merged_summary_op = tf.summary.merge_all()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    # Set the logs writer to the folder /tmp/tensorflow_logs
    summary_writer = tf.summary.FileWriter('logs2/', graph=sess.graph)

    # Training cycle
    for iteration in range(training_iteration):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys})
            # Compute the average loss
            avg_cost += sess.run(cost_function, feed_dict={x: batch_xs, y: batch_ys})/total_batch
            # Write logs for each iteration
            summary_str = sess.run(merged_summary_op, feed_dict={x: batch_xs, y: batch_ys})
            summary_writer.add_summary(summary_str, iteration*total_batch + i)
        # Display logs per iteration step
        if iteration % display_step == 0:
            print("Iteration:", '%04d' % (iteration + 1), "cost=", "{:.9f}".format(avg_cost))

    print("Tuning completed!")
    # Test the model
    predictions = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(predictions, "float"))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

Iteration: 0001 cost= 30.921934754
Iteration: 0003 cost= 21.002441988
Iteration: 0005 cost= 20.106136221
Iteration: 0007 cost= 19.674417160
Iteration: 0009 cost= 19.407800537
Iteration: 0011 cost= 19.174449246
Iteration: 0013 cost= 18.911463337
Iteration: 0015 cost= 18.731687451
Iteration: 0017 cost= 18.599568931
Iteration: 0019 cost= 18.518150640
Iteration: 0021 cost= 18.372092344
Iteration: 0023 cost= 18.452477425
Iteration: 0025 cost= 18.295915805
Iteration: 0027 cost= 18.201276648
Iteration: 0029 cost= 18.189183418
Tuning completed!
Accuracy: 0.924


In [1]:
#Launching TensorBoard
#To run TensorBoard, use the following command (alternatively python -m tensorboard.main)

#tensorboard --logdir=path/to/log-directory

In [2]:
#Once TensorBoard is running, navigate your web browser to localhost:6006 to view the TensorBoard.